## Index ##
[Importing modules](#Importing-required-modules)
<br>
[Importing data](#Importing-file-and-unzipping)
<br>
[Cleaning](#Cleaning)

### Importing required modules ###

* `requests` used for importing the downloaded zip file
* `tqdm` for progress bars, file size, elapsed time and download speeds
* `zipfile` for opening the zip file
* `concurrent.futures` for multiprocessing
* `pandas` as datatype

In [3]:
import requests
from tqdm.auto import tqdm
import zipfile
import concurrent.futures
import pandas as pd

### Importing file and unzipping ###

In [3]:
#Download data to hospitalPriceData.zip in working directory
hospitalPriceData_zip_url = 'https://www.dolthub.com/csv/dolthub/hospital-price-transparency-v3/iptu80riko4il5qij5asr8nalodeo9ut?include_bom=0'

#session to efficiently use data streaming
session = requests.Session()
response = session.get(hospitalPriceData_zip_url, stream= True)
total_size_in_bytes = int(response.headers.get('content-length', 0))
block_size = 1024**2 #1 MiB blockst
progress_bar = tqdm(total= total_size_in_bytes, unit= 'B', unit_scale= True)
with open("hospitalPriceData.zip", "wb") as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()

#2.31 GB, TOOK 1hr 15min WITH 675mb ETHERNET (though dolthub was the bottleneck)

2.31GB [1:15:14, 511kB/s] 


In [10]:
#%%time

#Extract files from hospitalPriceData.zip
zf = zipfile.ZipFile('hospitalPriceData.zip')
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(zf.extractall(), zf.infolist())

#3min 18s wall time normal
#3min 7s wall time multithreaded

CPU times: user 1min 19s, sys: 35.9 s, total: 1min 55s
Wall time: 3min 7s


### Cleaning ###